a) Formulación del problema de programación lineal

    Variables de decisión:
    x1 = número de unidades sencillas
    x2 = número de unidades dobles  
    x3 = número de unidades triples
    x4 = número de unidades cuádruples
    
    Función objetivo: Maximizar recaudación de impuestos
    Max Z = 1000*x1 + 1900*x2 + 2700*x3 + 3400*x4
    
    Restricciones:
    1. Área disponible: 0.18*x1 + 0.28*x2 + 0.4*x3 + 0.5*x4 <= 63.75 acres
       (300 casas * 0.25 acres = 75 acres, menos 15% para calles = 63.75 acres)
    
    2. Presupuesto: 50000*x1 + 70000*x2 + 130000*x3 + 160000*x4 <= 15000000
    
    3. Unidades triples y cuádruples >= 25% del total:
       x3 + x4 >= 0.25*(x1 + x2 + x3 + x4)
    
    4. Unidades sencillas >= 20% del total:
       x1 >= 0.20*(x1 + x2 + x3 + x4)
    
    5. Unidades dobles >= 10% del total:
       x2 >= 0.10*(x1 + x2 + x3 + x4)
    
    6. No negatividad: x1, x2, x3, x4 >= 0

b) Código con variables continuas en Julia (JuMP)

In [3]:
using JuMP, GLPK

In [5]:
model = Model(GLPK.Optimizer)

@variable(model, x1 >= 0)  # Unifamiliar
@variable(model, x2 >= 0)  # Doble
@variable(model, x3 >= 0)  # Triple
@variable(model, x4 >= 0)  # Cuádruple

# Función objetivo: maximizar ingresos fiscales
@objective(model, Max, 1000x1 + 1900x2 + 2700x3 + 3400x4)

# Restricción de área disponible
@constraint(model, 0.18x1 + 0.28x2 + 0.4x3 + 0.5x4 <= 63.75)

# Restricción de presupuesto
@constraint(model, 50000x1 + 70000x2 + 130000x3 + 160000x4 <= 15_000_000)

# Proporción de unidades triples y cuádruples >= 25%
@constraint(model, 3(x3 + x4) - (x1 + x2 + x3 + x4) >= 0)

# Proporción de unidades sencillas >= 20%
@constraint(model, 4x1 - (x1 + x2 + x3 + x4) >= 0)

# Proporción de unidades dobles >= 10%
@constraint(model, 10x2 - (x1 + x2 + x3 + x4) >= 0)

-x1 + 9 x2 - x3 - x4 >= 0

In [6]:

optimize!(model)

In [7]:
model

A JuMP Model
├ solver: GLPK
├ objective_sense: MAX_SENSE
│ └ objective_function_type: AffExpr
├ num_variables: 4
├ num_constraints: 9
│ ├ AffExpr in MOI.GreaterThan{Float64}: 3
│ ├ AffExpr in MOI.LessThan{Float64}: 2
│ └ VariableRef in MOI.GreaterThan{Float64}: 4
└ Names registered in the model
  └ :x1, :x2, :x3, :x4

In [8]:
println("Estado: ", termination_status(model))
println("Ingresos fiscales máximos: \$", objective_value(model))
println("Unidades sencillas: ", value(x1))
println("Unidades dobles: ", value(x2))
println("Unidades triples: ", value(x3))
println("Unidades cuádruples: ", value(x4))

Estado: OPTIMAL
Ingresos fiscales máximos: $343421.0526315791
Unidades sencillas: 39.47368421052633
Unidades dobles: 65.7894736842105
Unidades triples: 0.0
Unidades cuádruples: 52.63157894736848


c) Código con variables enteras (optimización entera)

In [9]:
using JuMP, GLPK

In [10]:
model = Model(GLPK.Optimizer)

@variable(model, x1 >= 0, Int)
@variable(model, x2 >= 0, Int)
@variable(model, x3 >= 0, Int)
@variable(model, x4 >= 0, Int)

@objective(model, Max, 1000x1 + 1900x2 + 2700x3 + 3400x4)

@constraint(model, 0.18x1 + 0.28x2 + 0.4x3 + 0.5x4 <= 63.75)
@constraint(model, 50000x1 + 70000x2 + 130000x3 + 160000x4 <= 15_000_000)
@constraint(model, 3(x3 + x4) - (x1 + x2 + x3 + x4) >= 0)
@constraint(model, 4x1 - (x1 + x2 + x3 + x4) >= 0)
@constraint(model, 10x2 - (x1 + x2 + x3 + x4) >= 0)

optimize!(model)

println("Estado: ", termination_status(model))
println("Ingresos fiscales máximos (enteros): \$", objective_value(model))
println("Unidades sencillas: ", value(x1))
println("Unidades dobles: ", value(x2))
println("Unidades triples: ", value(x3))
println("Unidades cuádruples: ", value(x4))

Estado: OPTIMAL
Ingresos fiscales máximos (enteros): $343000.0
Unidades sencillas: 40.0
Unidades dobles: 65.0
Unidades triples: 1.0
Unidades cuádruples: 52.0
